In [1]:
import pandas as pd

In [3]:
data = pd.read_csv('./data/15_best_features.csv')
data.drop('Unnamed: 0', axis=1, inplace=True)

In [4]:
data

,MinEStateIndex,BCUT2D_MRHI,PEOE_VSA12,PEOE_VSA6,SMR_VSA2,SMR_VSA6,SlogP_VSA8,VSA_EState5,NumAliphaticCarbocycles,NumAromaticCarbocycles,NumAromaticRings,fr_Ar_N,fr_Ar_NH,fr_aniline,fr_imidazole
0,0.352055,0.066227,0.204508,0.272727,0.0,0.126423,0.394180,0.686541,0.0,0.75,0.571429,0.250,0.000000,0.333333,0.0
1,0.835580,0.034018,0.204508,0.363636,0.0,0.075586,0.503610,0.706691,0.0,0.50,0.714286,0.750,0.333333,0.000000,1.0
2,0.858769,0.097317,0.204508,0.537485,0.0,0.075586,0.503610,0.746837,0.0,0.50,0.714286,0.750,0.333333,0.000000,1.0
3,0.951905,0.033767,0.204508,0.000000,0.0,0.142166,0.300139,0.758882,0.0,0.25,0.571429,0.625,0.333333,0.333333,0.0
4,0.484391,0.281136,0.204508,0.363636,0.0,0.423601,0.101045,0.747679,0.0,0.50,0.571429,0.375,0.000000,0.666667,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3266,0.747251,0.096757,0.609759,0.272439,0.0,0.720519,0.000000,0.702194,0.0,0.25,0.285714,0.250,0.000000,0.666667,0.0
3267,0.520731,0.096757,0.609759,0.272439,0.0,0.720519,0.000000,0.557935,0.0,0.25,0.285714,0.250,0.000000,0.666667,0.0
3268,0.853204,0.058713,0.407601,0.280409,0.0,0.557840,0.197421,0.705480,0.0,0.50,0.571429,0.375,0.000000,0.666667,0.0
3269,0.913943,0.048347,0.204508,0.181818,0.0,0.146935,0.394180,0.747912,0.0,0.50,0.428571,0.250,0.000000,0.166667,0.0


In [5]:
target = pd.read_csv('./data/jak3_activity.csv')
target   

,activity
0,1
1,1
2,1
3,0
4,1
...,...
3266,1
3267,1
3268,1
3269,1


In [11]:
target =target.to_numpy()
target

array([[1],
       [1],
       [1],
       ...,
       [1],
       [1],
       [1]])

In [12]:
data = data.to_numpy()
data

array([[0.35205508, 0.0662274 , 0.20450794, ..., 0.        , 0.33333333,
        0.        ],
       [0.83557986, 0.03401767, 0.20450794, ..., 0.33333333, 0.        ,
        1.        ],
       [0.85876856, 0.0973166 , 0.20450794, ..., 0.33333333, 0.        ,
        1.        ],
       ...,
       [0.85320432, 0.05871255, 0.40760079, ..., 0.        , 0.66666667,
        0.        ],
       [0.91394328, 0.04834692, 0.20450794, ..., 0.        , 0.16666667,
        0.        ],
       [0.85177675, 0.05871862, 0.40760079, ..., 0.        , 0.66666667,
        0.        ]])

In [19]:
#prepare data
import torch

import torch.nn as nn
import numpy as np
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

#prepare data

X, y = data, target

n_samples, n_features = X.shape

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


#scale
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

#convert to tensors

X_train = torch.from_numpy(X_train.astype(np.float32))
X_test = torch.from_numpy(X_test.astype(np.float32))
y_train = torch.from_numpy(y_train.astype(np.float32))
y_test = torch.from_numpy(y_test.astype(np.float32))

#reshape
y_train = y_train.view(y_train.shape[0], 1) #since we converted it to a tensor, it is in form of a row, We have to convert it  back ti a column
y_test = y_test.view(y_test.shape[0], 1)

#create model

class LogisticRegression(nn.Module):
    def __init__(self, n_input_features):
        super(LogisticRegression, self).__init__()
        #define layers
        self.lin = nn.Linear(n_input_features, 1)

    def forward(self, x):
        return torch.sigmoid(self.lin(x))


model = LogisticRegression(n_features)

#loss
criterion = nn.BCELoss()

#optimizer

optimizer = torch.optim.SGD(model.parameters(), lr=0.01)


#training loop 
num_epochs = 100000

for epoch in range(num_epochs):
    #forward pass and loss
    y_predicted = model(X_train)
    loss = criterion(y_predicted, y_train)

    #backward pass
    loss.backward()

    #update
    optimizer.step()

    #zero gradients
    optimizer.zero_grad()

    if (epoch+1) % 100 == 0:
        print(f'epoch: {epoch+1}, loss = {loss.item():.4f}')


with torch.no_grad():
    y_predicted = model(X_test)
    y_predicted_cls = y_predicted.round()
    acc = y_predicted_cls.eq(y_test).sum() / float(y_test.shape[0])
    print(f'accuracy = {acc:.4f}')
#plot


epoch: 100, loss = 0.6772
epoch: 200, loss = 0.6491
epoch: 300, loss = 0.6360
epoch: 400, loss = 0.6285
epoch: 500, loss = 0.6233
epoch: 600, loss = 0.6194
epoch: 700, loss = 0.6163
epoch: 800, loss = 0.6139
epoch: 900, loss = 0.6118
epoch: 1000, loss = 0.6101
epoch: 1100, loss = 0.6086
epoch: 1200, loss = 0.6073
epoch: 1300, loss = 0.6061
epoch: 1400, loss = 0.6051
epoch: 1500, loss = 0.6042
epoch: 1600, loss = 0.6033
epoch: 1700, loss = 0.6026
epoch: 1800, loss = 0.6019
epoch: 1900, loss = 0.6013
epoch: 2000, loss = 0.6007
epoch: 2100, loss = 0.6002
epoch: 2200, loss = 0.5997
epoch: 2300, loss = 0.5993
epoch: 2400, loss = 0.5988
epoch: 2500, loss = 0.5984
epoch: 2600, loss = 0.5981
epoch: 2700, loss = 0.5977
epoch: 2800, loss = 0.5974
epoch: 2900, loss = 0.5971
epoch: 3000, loss = 0.5968
epoch: 3100, loss = 0.5966
epoch: 3200, loss = 0.5963
epoch: 3300, loss = 0.5961
epoch: 3400, loss = 0.5959
epoch: 3500, loss = 0.5957
epoch: 3600, loss = 0.5955
epoch: 3700, loss = 0.5953
epoch: 380